## Import Libraries

In [1]:
import csv
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Read Config File

In [2]:
import configparser 
config = configparser.ConfigParser()
config.read('config.ini')
ip = config['DEFAULT']['IP']
port = config['DEFAULT']['MongoDB-Port']
db_name = config['DEFAULT']['DB-Name']
contain_string = config['DEFAULT']['Contain-String']
au_geo_file = config['DEFAULT']['AU-Geo-File']
world_geo_file = config['DEFAULT']['World-Geo-File']

## Connect MongoDB

In [3]:
import pymongo
from pymongo import MongoClient
client = MongoClient(ip, int(port))
db = client[db_name]

## Get Collections list

In [4]:
collection_name_list = db.collection_names()
collections_list = []

for i in collection_name_list:
    if contain_string in i:
        collections_list.append(i)
        
collections_list.sort()

## Supporting Functions

In [7]:
def get_geo_dic(file):
    geo_dic = {}
    with open(file, mode='r') as infile:
        reader = csv.reader(infile)
        for index, row in enumerate(reader):
            if index > 0:
                try:
#                     geo_dic[row[0]] = {"city":row[1],"state":row[2],"country":row[3]}
                    geo_dic[row[0]] = [row[1],row[2],row[3]]
                except Exception as e:
                    print("----------------------")
                    print("Error message: {}".format(e))
                    print("Error row: {}".format(row))
    return geo_dic

In [8]:
def insert_geoname(db,collection,geo_dic):
    for i in db[collection].find():
        uloc = i["user"]["location"]
        if (i["lang"] == "en") and (uloc is not None):
            try:
                if geo_dic[uloc] != ["","",""]:
#                     db[collection].update({"_id": i["_id"]}, {"$set": {"geoname": geo_dic[uloc]}})
                  db[collection].update({"_id": i["_id"]}, {"$set": {"geoname": {"city":geo_dic[uloc][0],"state":geo_dic[uloc][1],"country":geo_dic[uloc][2]}}})  
            except:
                print(uloc)

## Get Geoname Dictionary (Australia & Other)

In [9]:
au_geo_dic = get_geo_dic(au_geo_file)
world_geo_dic = get_geo_dic(world_geo_file)

## Insert Geoname in MongoDB

In [11]:
# insert one by one
for collection in collections_list:
    if "Australia" in collection:
        insert_geoname(db,collection,au_geo_dic)
    else:
        insert_geoname(db,collection,world_geo_dic)
    print("Insertion for " + collection + " is finished")

41.65339493751526
